## Functions (run first, do not update)

In [ ]:
import os
import subprocess
import json
import numpy as np
from PIL import Image

def extract_flir_tags(image_path):
    """
    If using FLIR need to extract calibration constants determined uniquely for each camera sensor to convert their radiance unit to temperature for corrections.

    Parameters:
    image_path (str): Path to one image file from FLIR camera at site.

    Returns:
    dictionary: Contains FLIR tags 'PlanckR1', 'PlanckB', 'PlanckF', 'PlanckR2', 'RawThermalImage', 'RawThermalImageWidth', 'RawThermalImageHeight'.

    Note: Constants will need to be updated if camera is re-calibrated, need to download exiftool and have installed on computer https://exiftool.org/
    """
    command = ['exiftool', '-G', '-j', image_path]
    result = subprocess.run(command, capture_output=True, text=True)
    metadata = json.loads(result.stdout)[0] 
    desired_tags = ['FLIR:PlanckR1', 'FLIR:PlanckB', 'FLIR:PlanckF', 'FLIR:PlanckR2', 'FLIR:PlanckO','FLIR:RawThermalImage', 'FLIR:RawThermalImageWidth', 'FLIR:RawThermalImageHeight']

    
    extracted_data = {}
    for tag in desired_tags:
        if tag in metadata:
            key = tag.split(':')[1]  # Extract the part after the colon
            extracted_data[key] = metadata[tag]

    raw_command = ['exiftool', '-b', '-RawThermalImage', image_path]
    raw_result = subprocess.run(raw_command, capture_output=True)
    raw_data_bytes = raw_result.stdout
    raw_data_array = np.frombuffer(raw_data_bytes, dtype=np.uint16)

    header_length = 102  # LENGTH OF HEADER
    image_data = raw_data_array[header_length:]

    try:
        # Attempt to reshape
        image_data = image_data.reshape((480, 640))
        # If no errors, you've likely removed the header
    except ValueError:
        print(f"Header removal of {header_length} bytes failed.")

    extracted_data['RawThermalImageData'] = image_data

    return extracted_data

def FLIR_to_temp_vectorized(flir_tags_dict, raw_image):
    """
    Convert the raw signal of the entire image to uncorrected temperature (Kelvin) using vectorized operations.

    Parameters:
    flir_tags_dict (dict): Dictionary containing FLIR tags.
    raw_image (numpy.ndarray): 2D array containing raw signal values.

    Returns:
    numpy.ndarray: 2D array with temperature values in Kelvin.
    """
    R1 = flir_tags_dict["PlanckR1"]
    B = flir_tags_dict["PlanckB"]
    F = flir_tags_dict["PlanckF"]
    R2 = flir_tags_dict["PlanckR2"]
    O = flir_tags_dict["PlanckO"]

    temp_kelvin = B / np.log(R1 / (R2 * (raw_image + O)) + F)
    return temp_kelvin

def process_and_save_thermal_image_optimized(raw_image_path, output_path, csv_path=None):
    """
    Process the raw thermal image to convert it to temperature values and save as a .tiff.

    Parameters:
    flir_tags_dict (dict): Dictionary containing FLIR tags.
    raw_image_path (str): Path to the raw thermal image.
    output_path (str): Path where the processed .tiff image should be saved.
    """
    # Extract the raw thermal data and dimensions
    extracted_data = extract_flir_tags(raw_image_path)
    raw_data = extracted_data["RawThermalImageData"]
    width = extracted_data["RawThermalImageWidth"]
    height = extracted_data["RawThermalImageHeight"]
    
    # Reshape the raw data
    raw_image = raw_data.reshape(height, width)

    # Convert raw values to temperature values using vectorized function
    temperature_image = FLIR_to_temp_vectorized(extracted_data, raw_image)
    if csv_path:
        np.savetxt(csv_path, temperature_image, delimiter=',')
    
    # Convert Kelvin to Celsius directly using numpy and save
    temperature_image_celsius = temperature_image - 273.15

    # Convert temperature values to an appropriate dtype (e.g., float32) for saving as TIFF
    temperature_image_celsius = temperature_image_celsius.astype(np.float32)

    # Save the processed data as a .tiff image
    Image.fromarray(temperature_image_celsius).save(output_path)

    pass

## .seq files to .tiff files processing
### Update file paths
### Need to download exiftool @ https://exiftool.org/

In [ ]:
#Assumes you have a folder for each month and running one year at a time
base_dir = 'path/to/seq/image/folders'
output_base_dir = 'path/to/save/tiff/images'

# Iterate through each month's folder
for month in os.listdir(base_dir):
    month_dir = os.path.join(base_dir, month)
    # Check if it's a directory
    if os.path.isdir(month_dir):
        # Create a corresponding directory in the output structure
        output_month_dir = os.path.join(output_base_dir, month)
        os.makedirs(output_month_dir, exist_ok=True)  # This will create the directory if it doesn't exist, without raising an error if it does exist
        
        # Process each image in the month's folder
        for image_name in os.listdir(month_dir):
            if image_name.lower().endswith('.seq'):  # Check for raw image file extension
                raw_image_path = os.path.join(month_dir, image_name)
                output_image_name = image_name.rsplit('.', 1)[0] + '.tiff'
                output_path = os.path.join(output_month_dir, output_image_name)
                
                # Skip processing if the .tiff file already exists
                if os.path.exists(output_path):
                    print(f"Skipped {raw_image_path}, output already exists.")
                    continue
                
                try:
                    print(f"Processing {raw_image_path}")
                    # Process the image and save it
                    process_and_save_thermal_image_optimized(raw_image_path, output_path)
                except Exception as e:
                    print(f"Failed to process {raw_image_path}. Error: {e}")
                    # You can add any additional error handling here if needed